In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
# Pkg.add("PyPlot")
# Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random
using Plots
# using PyPlot
# using PyCall

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.99
betas = [0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999, 0.99999]
objectiv_values = []
exp_profits = []
CVaR_values = []

Any[]

In [5]:
for beta in betas
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W] >= 0)

    # Define the objective function
    @objective(model, Max, sum(1/W .*
    (scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
    + beta  * (zeta - (1/(1-alpha)) * sum(1/W .* eta[i] for i in 1:W)))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)


    # Solve the optimization problem
    optimize!(model)

    exp_profit = sum(1/W .*
    (scenarios[i][hour,"price"] * value.(p_DA[hour])
    + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)

    CVaR = beta  * (value.(zeta) - (1/(1-alpha)) * sum(1/W .* value.(eta[i]) for i in 1:W))

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Optimal solution found")
        
        # RETURN OBJECTIVE value
        #println("Objective value: ", objective_value(model))
        #println("Expected profit: ", exp_profit)
        #println("CVaR: ", CVaR)
        push!(objectiv_values, objective_value(model))
        push!(exp_profits, exp_profit)
        push!(CVaR_values, CVaR)
    else
        println("No optimal solution found")
    end
end

Optimal solution found
Objective value: 154391.55537235324
Expected profit: 151705.65829751402
CVaR: 2685.897074837732
Optimal solution found
Objective value: 155734.50390977206
Expected profit: 151705.65829751402
CVaR: 4028.8456122565976
Optimal solution found
Objective value: 156540.27303222337
Expected profit: 151705.65829751402
CVaR: 4834.614734707917
Optimal solution found
Objective value: 156808.86273970714
Expected profit: 151705.65829751402
CVaR: 5103.20444219169
Optimal solution found
Objective value: 157023.7345056942
Expected profit: 151705.65829751402
CVaR: 5318.076208178709
Optimal solution found
Objective value: 157072.08065304125
Expected profit: 151705.65829751402
CVaR: 5366.422355525789
Optimal solution found
Objective value: 157076.91526777597
Expected profit: 151705.65829751402
CVaR: 5371.256970260496
Optimal solution found
Objective value: 157077.39872924946
Expected profit: 151705.65829751402
CVaR: 5371.740431733967


In [20]:
#make a dataframe with the results
results = DataFrame(betas = betas, objective_values = objectiv_values, exp_profits = exp_profits, CVaR_values = CVaR_values)

#save the results
CSV.write("1_3_results/one_price_results.csv", results)

"1_3_results/one_price_results.csv"